# Week 03 - AutoFix

This week, you will delve into the practical application of large language models (LLMs) to automate **code remediation**, a key challenge in modern DevOps. You will implement an “AutoFix” system that automatically proposes fixes for issues identified by a language server.

## Goal: Automated Issue Resolution
Your primary task is to implement an `auto_fix` function that serves as the core of this system. This function will act as a bridge between **diagnostic output** and **code changes**, using an LLM to generate a precise solution.

**Input**: An error object returned by the language server (e.g. a type-checking failure).

**Output**: A proposed change in unified diff format to resolve the issue.

Focus on designing an effective LLM prompt and context strategy that reliably converts a structured error message into a correct and syntactically valid unified diff.

## Environment Variables & Imports

Use the following cells to add any needed environment variables (like API keys) **before** loading any of the python modules.

In [ ]:
import initialize_notebook # noqa

In [ ]:
import pathlib
import random
import re

import jinja2

from hslu.dlm03.common import backend as backend_lib
from hslu.dlm03.tools import lint
from hslu.dlm03.util import ipython_utils, unified_diff

## AutoFix Implementation

In [ ]:
BACKEND_CONFIG = backend_lib.Gemini2p5Flash()
BACKEND = BACKEND_CONFIG.get_backend()

In [ ]:
AUTO_FIX_PROMPT_TEMPLATE = """[Role]
You are an AI Expert Software Engineer.

[Task]
You will be provided with a code file as well as an error from a static code analysis tool, and you should provide a fix for this error.

[Output]
For each file that needs to be changed, write out the changes similar to a unified diff like `diff -U0` would produce.

Return edits similar to unified diffs that `diff -U0` would produce.

Make sure you include the first 2 lines with the file paths.
Don't include timestamps with the file paths.
Please ensure the line numbers are correct.

Start each hunk of changes with a `@@ ... @@` line.

The user's patch tool needs CORRECT patches that apply cleanly against the current contents of the file!
Think carefully and make sure you include and mark all lines that need to be removed or changed as `-` lines.
Make sure you mark all new or modified lines with `+`.
Don't leave out any lines or the diff patch won't apply correctly.

Indentation matters in the diffs!

Start a new hunk for each section of the file that needs changes.

Only output hunks that specify changes with `+` or `-` lines.
Skip any hunks that are entirely unchanging ` ` lines.

Output hunks in whatever order makes the most sense.
Hunks don't need to be in any particular order.

When editing a function, method, loop, etc use a hunk to replace the *entire* code block.
Delete the entire existing version with `-` lines and then add a new, updated version with `+` lines.
This will help you generate correct code and correct diffs.

To move code within a file, use 2 hunks: 1 to delete it from its current location, 1 to insert it in the new location.

To make a new file, show a diff from `--- /dev/null` to `+++ path/to/new/file.ext`.

[Code]
{% for filename, content in files.items() %}
# {{ filename }}:
{{ content }}

{% endfor %}

[Issue]
Issue file: {{ issue.filename }}
Issue Line: {{ files[issue.filename].splitlines()[issue.location.row - 1] }}
Issue Code: {{ issue.code }}
Issue Message: {{ issue.message }}
"""
AUTO_FIX_PROMPT = jinja2.Template(AUTO_FIX_PROMPT_TEMPLATE, undefined=jinja2.StrictUndefined)

In [ ]:
def auto_fix(backend: backend_lib.LLMBackend, issue: lint.Issue,
             auto_fix_prompt_template: jinja2.Template,
             retry: int = 5) -> unified_diff.UnifiedDiff:
    file_path = pathlib.Path(issue.filename)
    code = file_path.read_text()
    prompt = auto_fix_prompt_template.render(files={issue.filename: code}, issue=issue)

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": ""},
    ]

    response = None
    exceptions = []
    while response is None:
        try:
            response = backend(messages=messages)
        except Exception as e:
            retry -= 1
            exceptions.append(e)
            if retry < 0:
                raise ExceptionGroup("Retry failed.", exceptions)

    choice = random.choice(response.choices)
    diff = choice.message.content
    if code_block := re.match(r"```([ a-z]*)?\n(.*)?```", diff, re.DOTALL):
        diff = code_block.group(2).strip()
    fix_diff = unified_diff.UnifiedDiff.from_string(diff)
    return fix_diff

In [ ]:
LINT_PATH = pathlib.Path("/Users/vincent/Development/Valinor/valinor/hslu/dlm03/**/*.py")
ipython_utils.display_autofix(lambda: lint.lint(LINT_PATH), lambda issue: auto_fix(BACKEND, issue, AUTO_FIX_PROMPT), None,
                              False)